# Lemmatization and full text search (FTS)


## Solution of https://github.com/apohllo/nlp/blob/master/2-fts.md

In [2]:
from elasticsearch import Elasticsearch
from os import listdir
from os.path import isfile, join

In [3]:
client = Elasticsearch([
    {'host': 'localhost'},
])

In [75]:
    create_index_body = {
      'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 0,

        'analysis': {
          'analyzer': {
            'bill_analyzer': {
              'type': 'custom',
              'tokenizer': 'standard',
              'filter': ['synonym', 'morfologik_stem', 'lowercase']
            }
          },
        "filter": {
        "synonym": {
          "type": "synonym",
          "lenient": True,
          "synonyms": [
            "kpk, kodeks postępowania karnego",
            "kpc, kodeks postępowania cywilnego",
            "kk, kodeks karny",
            "kc, kodeks cywilny"
          ]
        }
      }
        }
      },
      'mappings': {
        'bill': {
          'properties': {
            'content': {'type': 'text', 'analyzer': 'bill_analyzer'},
          }
        }
      }
    }

In [76]:
client.indices.create(    
    index='bill_index1',
     body=create_index_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bill_index1'}

In [77]:
def load_data(directory_path):
    files = [file for file in listdir(directory_path) if isfile(join(directory_path, file))]
    for file in files:
        with open(f"{directory_path}/{file}") as input_file:
            client.index(index='bill_index1', id=f"{directory_path}/{file}", doc_type='bill', body={'content':input_file.read() })

In [78]:
load_data('./data/')

In [31]:
client.count(index='bill_index1', body = { "query": {
        "query_string": {
            "query": "ustawa"
        }
    }})

{'count': 1179,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [32]:
client.count(index='bill_index1', body = { "query": {
        "query_string": {
            "query": "\"kpc\""
        }
    }})

{'count': 100,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [33]:
client.count(index='bill_index1', body = {"query": {
        "match_phrase" : {
            "content" : {
              "query": "wchodzi w życie",
              "slop": 2
            }
        }
    }})

{'count': 1175,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [35]:
result =client.search(index='bill_index1', body = {
    "query": {
        "query_string": {
            "query": "konstytucja"
        }
    }
})

In [48]:
top_ten = result['hits']['hits'][0:10]

In [50]:
top_ten[0]

{'_index': 'bill_index1',
 '_type': 'bill',
 '_id': './data//1997_629.txt',
 '_score': 6.871392,
 '_source': {'content': '\n\n\n\nDz.U. z 1994 r. Nr 61, poz. 251\n                         USTAWA  KONSTYTUCYJNA\n                       z dnia 22 kwietnia 1994 r.\n                                    \n         o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia Konstytucji Rzeczypospolitej Polskiej\n                                    \n                                    \n                                Art. 1.\nW ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i \nuchwalenia Konstytucji Rzeczypospolitej Polskiej (Dz.U. Nr 67, poz. 336) \nwprowadza się następujące zmiany:\n \n 1) po art. 2 dodaje się art. 2a, art. 2b i art. 2c w brzmieniu:\n   "Art. 2a. 1. Inicjatywa ustawodawcza w zakresie przedstawienia\n                Zgromadzeniu Narodowemu projektu nowej Konstytucji\n                przysługuje również grupie obywateli, którzy

In [59]:
def get_year_position(file_name):
    last_part = file_name.split("/")[-1]
    only_name = last_part.split(".")[0]
    year, position = only_name.split("_")
    return year, position

In [61]:
res_list = []
for result in top_ten:
    year, position = get_year_position(result['_id'])
    res_list.append((result['_score'], year, position ))

In [63]:
for res in res_list:
    print(f"score: {res[0]}, year: {res[1]} position:{res[2]} ")

score: 6.871392, year: 1997 position:629 
score: 6.666975, year: 2000 position:443 
score: 6.6366324, year: 1997 position:604 
score: 6.631108, year: 1996 position:350 
score: 6.2556105, year: 1997 position:642 
score: 6.0630817, year: 2001 position:23 
score: 5.9332466, year: 1996 position:199 
score: 5.854491, year: 1999 position:688 
score: 5.4713526, year: 2001 position:1082 
score: 5.4713526, year: 1997 position:681 


In [74]:
client.search(index='bill_index1', body = {
    "query": {
        "query_string": {
            "query": "konstytucja"
        }
            },
       "highlight" : {
        "number_of_fragments" : 3,
        "fields" : {
            "content" : {}
        }
    }
})

{'took': 147,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': 45,
  'max_score': 6.871392,
  'hits': [{'_index': 'bill_index1',
    '_type': 'bill',
    '_id': './data//1997_629.txt',
    '_score': 6.871392,
    '_source': {'content': '\n\n\n\nDz.U. z 1994 r. Nr 61, poz. 251\n                         USTAWA  KONSTYTUCYJNA\n                       z dnia 22 kwietnia 1994 r.\n                                    \n         o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia Konstytucji Rzeczypospolitej Polskiej\n                                    \n                                    \n                                Art. 1.\nW ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i \nuchwalenia Konstytucji Rzeczypospolitej Polskiej (Dz.U. Nr 67, poz. 336) \nwprowadza się następujące zmiany:\n \n 1) po art. 2 dodaje się art. 2a, art. 2b i art. 2c w brzmieniu:\n   "Art. 2a. 1. Ini